<a href="https://colab.research.google.com/gist/oserikov/3c6e277e4da64404e898ace122b65b70/openapi-codegen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/deepmipt/DeepPavlov.git@openapi_integration docstring_parser  pytablereader 

In [2]:
import re
from deeppavlov.core.data.utils import download_decompress
from docstring_parser import parse
import pytablereader as ptr
import requests
from collections import defaultdict
import importlib.util
import sys
import os
import inspect
from pprint import pprint

In [3]:
# pass the desired openapi endpoint
OPENAPI_URL = "https://api.apis.guru/v2/specs/jokes.one/1.1/swagger.json"

# swagger codegen to build python api for the openapi documented api
swagger_generator_endpoint = "https://generator.swagger.io/api/gen/clients/python"

# saving config
save_path = "save_path"
load_path = save_path

download codegenerated module

In [4]:
packageName = "openapi_client_codegen"
res = requests.post(swagger_generator_endpoint, json={"swaggerUrl": OPENAPI_URL,
                                                      "options": {"packageName": packageName}})

if str(res.status_code).startswith('2'):
    endpoint_sdk_zip_url = res.json().get("link")
    download_decompress(endpoint_sdk_zip_url, save_path, archive_type='zip')

2021-03-23 05:30:51.355 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from https://generator.swagger.io/api/gen/download/545680c5-729c-4144-9c27-2cbfeab7c66e to save_path/545680c5-729c-4144-9c27-2cbfeab7c66e
100%|██████████| 44.1k/44.1k [00:00<00:00, 2.20MB/s]
2021-03-23 05:30:51.457 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting save_path/545680c5-729c-4144-9c27-2cbfeab7c66e archive into save_path


read module docs and take all the api functions from here

In [5]:
md_path = f"{save_path}/python-client/README.md"

docs_md_tables_reader = ptr.TableFileLoader(md_path, format_name="markdown")
docs_md_tables = docs_md_tables_reader.load()
tables = []
for table in docs_md_tables:
    table = [dict(zip(table.headers, row)) for row in table.rows]
    tables.append(table)

func_name2class = defaultdict(list)
for table in tables:
    table_classes = {t["Class"] for t in table}
    for t in table:
        func_name2class[t["Method"]].append(t["Class"])

func_name2class = dict(func_name2class)

In [6]:
func_name2class

{'jod_categories_get': ['JodApi'],
 'jod_get': ['JodApi'],
 'joke_categories_search_get': ['JokeApi'],
 'joke_delete': ['JokeApi'],
 'joke_get': ['JokeApi'],
 'joke_list_get': ['JokeApi'],
 'joke_patch': ['JokeApi'],
 'joke_put': ['JokeApi'],
 'joke_random_get': ['JokeApi'],
 'joke_search_get': ['JokeApi'],
 'joke_tags_add_post': ['JokeApi'],
 'joke_tags_remove_post': ['JokeApi']}

parse module content to load all the API functions and have them available by name

In [7]:
plk_dir = f"{save_path}/python-client"
pkg_dir = f"{plk_dir}/{packageName}"
apimodule_name = "api"
api_dir = f"{plk_dir}/{apimodule_name}/"
sys.path = [plk_dir, pkg_dir] + sys.path

bad_params = ["async_req"]  # smth added by codegen api, this param complicates everything

api_module = importlib.import_module('api', package=packageName)
func_infos = dict()
for api_module_elem_name, api_module_elem in api_module.__dict__.items():
    if inspect.isclass(api_module_elem):
        api_module_submodule = api_module_elem()
        for func_name, func_val in inspect.getmembers(api_module_submodule):
            if func_name.startswith("__") and func_name.endswith("__"):
                # skip dunders cause they're unlikely to be apis
                continue
            if callable(func_val):
                func_doc = func_val.__doc__  # get the docstring from the codegenerated example
                if func_doc:
                    func_doc = re.sub(r'.+:param\s+async_req\s+bool.*', '', func_doc)  # remove codegenerated bad_param docstring
                func_doc_parsed = parse(func_doc)
                func_params = [param for param in func_doc_parsed.params if param not in bad_params]
                func_info = {
                    "module": api_module,
                    "func": func_val,
                    "params": func_params,
                    "params_di": [fp.__dict__ for fp in func_params]
                }
                func_infos[func_name] = func_info

In [8]:
some_funcs_to_take_a_look_at = {k: func_infos[k] for k_ix, k in enumerate(func_infos) if k_ix < 4}
pprint(some_funcs_to_take_a_look_at, width=120, compact=True)

{'jod_categories_get': {'func': <bound method JodApi.jod_categories_get of <openapi_client_codegen.api.jod_api.JodApi object at 0x7f2311b02cd0>>,
                        'module': <module 'api' from 'save_path/python-client/openapi_client_codegen/api/__init__.py'>,
                        'params': [],
                        'params_di': []},
 'jod_categories_get_with_http_info': {'func': <bound method JodApi.jod_categories_get_with_http_info of <openapi_client_codegen.api.jod_api.JodApi object at 0x7f2311b02cd0>>,
                                       'module': <module 'api' from 'save_path/python-client/openapi_client_codegen/api/__init__.py'>,
                                       'params': [],
                                       'params_di': []},
 'jod_get': {'func': <bound method JodApi.jod_get of <openapi_client_codegen.api.jod_api.JodApi object at 0x7f2311b02cd0>>,
             'module': <module 'api' from 'save_path/python-client/openapi_client_codegen/api/__init__.py'>,


In [9]:
func_infos["jod_get"]["func"](category="jod")

{'contents': {'copyright': '2019-20 https://jokes.one',
              'jokes': [{'background': '',
                         'category': 'jod',
                         'date': '2021-03-23',
                         'description': 'Joke of the day ',
                         'joke': {'clean': '0',
                                  'date': '2021-03-23',
                                  'id': 'I5ljXf1MiMj2Mix0T23CdQeF',
                                  'lang': 'en',
                                  'length': '57',
                                  'racial': '0',
                                  'text': 'What is wrong with Polish snow '
                                          'tires?\r\n'
                                          '\r\n'
                                          '\r\n'
                                          'They melt.\r\n'
                                          '\r\n',
                                  'title': 'What is wrong with Polish snow '
                